# Proyecto para resolver un problema empresarial complejo

# Objetivos del proyecto
Crear una hoja de calculos que servira como base de sistema de recomendaciones de cursos.

Primero hay que obtener los datos de la base de datos\
Para eso escribiremos un codigo SQL y descargaremos el resultado al DataFrame

In [ ]:
# Importamos las bibliotecas
import pandas as pd
import psycopg2
import psycopg2.extras 
import numpy as np
import itertools
from collections import Counter

In [ ]:
# Escribimos el codigo SQL y lo adjuntamos a la variable query
query = '''with joined as \
	(select \
	 	ca.user_id, \
	 	ci.resource_id, \
	 	ci.resource_type, \
	 	ca.state \
	from \
		final.carts as ca \
	join \
		final.cart_items as ci on ca.id=ci.cart_id\
	where \
		ci.resource_type='Course' and ca.state='successful' ),\
\
course_count as \
	(select \
		user_id, \
		count( distinct resource_id) as cnt \
	from joined \
	where \
		resource_type='Course' and state='successful' group by user_id) \
select \
	distinct(cc.user_id), \
	j.resource_id \
from course_count cc \
join \
	joined j on cc.user_id=j.user_id \
where \
	cc.cnt>1 \
order by 1'''.format()

In [ ]:
# Escribimos una funcion para obtener los datos de la base de datos y los guardamos en la variable df
def get_data():
    conn = psycopg2.connect(
        dbname='skillfactory',
        user='skillfactory',
        host='84.201.134.129',
        password='cCkxxLVrDE8EbvjueeMedPKt', 
        port='5432',
        )
    dict_cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
    dict_cur.execute(query)
    rows = dict_cur.fetchall()
    data = []
    for row in rows:
        data.append(dict(row))
    return data

df = pd.DataFrame(get_data())

In [ ]:
# Cramos la variable pair_courses_cnt en la cual vamos a guardar pares de los cursos comprados y la cantidad de veces estos cursos han sido comprados
# Agrupamos el dataframe
df_set = df.groupby('user_id')['resource_id'].agg(['nunique', set]).reset_index()

# Creamos la variable pairs, en la cual escribimos todas las combinaciones de pares de cursos posibles
pairs = df_set['set'].apply(lambda x: list(itertools.combinations(sorted(x), 2)))

# Creamos la variable pair_courses, en la cual escribimos los pares de cursos usando la funcion de ciclo
pair_courses = []
for course in pairs.values:
    for elements in course:
        pair_courses.append(elements)

# Con la funcion de Counter contamos la cantidad de pares. 
# Obtenemos el diccionario pair_courses_cnt
# En el cual la llave es el par de cursos y el significado es la cantidad de veces que ese par ha sido comprado
pair_courses_cnt = Counter(pair_courses)

In [ ]:
# Creamos el dataframe pair_df para contar el valor minimo de la frequencia de compra de el par de cursos 
pair_df = pd.DataFrame({'pair':pair_courses_cnt.keys(), 'value':pair_courses_cnt.values()})
pair_df = pair_df.sort_values(by='value', ascending=False)
pair_df.head(5)

,pair,value
186,"(551, 566)",797
381,"(515, 551)",417
205,"(489, 551)",311
105,"(523, 551)",304
519,"(566, 794)",290


In [ ]:
# En la variable min_freq escribimos el 75% de la frecuencia de compra. Esa sera nuestro limite de frecuencia bajo
# Ademas aplique la funcion "describe" para la columna de frequencia de compras para ver mejor los valores
# Eligi el 75% como limite porque con el limite mas alto, en la hoja de calculos resultada se borraban demasiados valores
min_freq = np.percentile(pair_df['value'], 75)
display(pair_df['value'].describe())
display(min_freq)

count    3989.000000
mean       10.031838
std        26.355998
min         1.000000
25%         1.000000
50%         3.000000
75%         9.000000
max       797.000000
Name: value, dtype: float64

9.0

In [ ]:
# Creamos la hoja de calculos con los identificadores de cursos unicos.
# De esa hoja vamos a sacar los valores y transferirlos a la funcion
courses_id = set()
for i in pair_courses:
    for elem in i:
        courses_id.add(elem)
unique_courses = pd.DataFrame(courses_id, columns=['course_id'])
unique_courses.head(5)

,course_id
0,513
1,514
2,515
3,516
4,517


In [ ]:
# Escribimos la funcion de la lista de recomendaciones

def recomendation(course_id):
    # Primero creamos el ciclo, que iterará sobre los valores de la variable pair_courses_cnt y los escribira en la variable freq
    # Ademas esa funcion va a comprobar 2 condiciones:
    # 1 Correspondencia del valor de ciclo y del valor transferido a la funcion
    # 2 Comprobar que la frequencia de id del curso es mayor que el limite de la frequencia bajo
    freq = []
    for i in pair_courses_cnt.keys():
        if i[0] == course_id and pair_courses_cnt[i] >= min_freq:
            freq.append((i, pair_courses_cnt[i]))
    
    # A continuacion ordenamos la lista freq en descendencia
    freq = sorted(freq, key=lambda x: x[1], reverse=True)
    
    # Mostramos las recomendaciones resultantes, que corresponden a las condiciones anteriores
    # Si la recomendacion no cumple las condidiones entonces recomendamos los 2 cursos mas populares
    if len(freq) >= 2:
        recom_1 = freq[0][0][1]
        recom_2 = freq[1][0][1]
    else: 
        recom_1 = 551
        recom_2 = 556
    return recom_1, recom_2

In [ ]:
# Aplicamos la funcion a la hoja unique_courses
unique_courses[['recom_1', 'recom_2']] = unique_courses['course_id'].apply(lambda x: pd.Series(recomendation(x)))
unique_courses.head(5)

,course_id,recom_1,recom_2
0,513,551,1141
1,514,551,515
2,515,551,523
3,516,745,553
4,517,551,750
